In [ ]:
from selenium import webdriver
import json

driver = webdriver.Chrome()
driver.get('https://www.tiktok.com/')

input('Once login is complete, press ENTER to continue')

# Save cookies to log in automatically
cookies = driver.get_cookies()
with open('tiktok_cookies.json', 'w') as file:
    json.dump(cookies, file)

print('TikTok cookies saved successfully')

In [ ]:
# RUN PROGRAM IN LOCAL ENVIRONMENT

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
import json
import pandas as pd

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Mobile Safari/537.36')
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.tiktok.com/@timnasindonesia/video/7485727263405722898?lang=en')

time.sleep(random.uniform(7, 10))

# Using cookies to log in automatically to the TikTok account
with open('tiktok_cookies.json', 'r') as file:
    cookies = json.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)
driver.refresh()

time.sleep(random.uniform(7, 10))

# Access comments
access_comments_button = driver.find_element(By.CSS_SELECTOR, 'g[filter="url(#Icon_Color-Comment_Shadow_svg__a)"]')
ActionChains(driver).move_to_element(access_comments_button).click().perform()

time.sleep(random.uniform(7, 10))

# Scrolling comments to load all comments
comment_container = driver.find_element(By.CLASS_NAME, 'css-147ti1k-DivCommentListContainer')
last_height = driver.execute_script('return arguments[0].scrollHeight;', comment_container)
comment_count = 0

while True:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', comment_container)
    
    if comment_count >= 1000:
        sleep_time = 15
    elif comment_count >= 500:
        sleep_time = 10
    else:
        sleep_time = 5

    time.sleep(sleep_time)
    
    new_height = driver.execute_script('return arguments[0].scrollHeight;', comment_container)
    
    if new_height == last_height:
        break
    last_height = new_height

    comment_count = len(driver.find_elements(
        By.XPATH, 
        '//p[@data-e2e="comment-level-1"][@class="css-1db1lf5-PCommentText emikstp6"]/span[1]',
    ))

print('Comment scrolling completed')

time.sleep(random.uniform(7, 10))

# Scraping comments
comments = driver.find_elements(
    By.XPATH, 
    '//p[@data-e2e="comment-level-1"][@class="css-1db1lf5-PCommentText emikstp6"]/span[1]',
)

# Save comments to list
comments_text = [comment.text for comment in comments]

# Set comments into dataframe
comments_df = pd.DataFrame(comments_text, columns=['comment'])
print(comments_df.shape)

Comment scrolling is complete
(683, 1)


In [4]:
# Show comments into dataframe
pd.set_option('display.max_colwidth', None)
comments_df.sample(5)

,comment
39,"emil duluu cobaaaaaa minn, debut in di gbk"
362,lah emilll
58,tandai gue kalau indonesia menang 3-1
485,🔥🔥
441,🤣


In [5]:
# Save comments into CSV format
comments_df.to_csv('tiktok_comments.csv', index=False, encoding='utf-8')